In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import ensemble, tree, linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Preprocessing the rows

In [39]:
housep_df = pd.read_csv("./Data/example.csv")

print(housep_df.iloc[4:5,:])
housep_df = housep_df.drop(['sold_price'], axis = 1)
#make a compy of the 1000th row of the dataframe
row_example = housep_df.iloc[4:5,:]
# row_example = housep_df.iloc[0:1,:].copy()

  price_per_m2  asking_price    price_evolution            typeProperty  \
4          NaN  2 395 000 kr  -95 000 kr (-4 %)  Vinterbonat fritidshus   

   typeOfForm  rooms   size bisize balkong tomtarea buildArea          cost  \
4  Äganderätt  2 rum  45 m²   7 m²     NaN   920 m²      1910  32 034 kr/år   

  uteplats floor                                               area  \
4      NaN   NaN  Vinterbonat fritidshus - Rådmansö - Södersvik,...   

     sold_price         type  
4  2 300 000 kr  bostadsratt  


In [92]:
housep_df.iloc[1:2,:]

,price_per_m2,asking_price,price_evolution,typeProperty,typeOfForm,rooms,size,bisize,balkong,tomtarea,buildArea,cost,uteplats,floor,area,type
1,NaN,1 695 000 kr,-195 000 kr (-12 %),Fritidshus,Äganderätt,3 rum,60 m²,NaN,NaN,3 390 m²,1952,31 000 kr/år,NaN,NaN,"Fritidshus - Grödinge, Botkyrka kommun - Såld ...",bostadsratt


In [41]:
row_example


,price_per_m2,asking_price,price_evolution,typeProperty,typeOfForm,rooms,size,bisize,balkong,tomtarea,buildArea,cost,uteplats,floor,area,type
4,NaN,2 395 000 kr,-95 000 kr (-4 %),Vinterbonat fritidshus,Äganderätt,2 rum,45 m²,7 m²,NaN,920 m²,1910,32 034 kr/år,NaN,NaN,"Vinterbonat fritidshus - Rådmansö - Södersvik,...",bostadsratt


In [42]:
!pip install geopy


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [43]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

def get_coordinates(location_name):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(location_name)
    if location:
        return location.latitude, location.longitude
    else:
        return None

def calculate_distance(origin, destination):
    origin_coords = get_coordinates(origin)
    destination_coords = get_coordinates(destination)

    if origin_coords and destination_coords:
        distance = geodesic(origin_coords, destination_coords).kilometers
        return distance
    else:
        return None
    


In [44]:
def pre_processing_precise_location(row_example):
    row_example['precise_location'] = ''
    a = row_example['area'].iloc[0]
    a = a.split('-')
    c= a[-2]
    c = c.split(' ')
    c = c[-4] + ' ' + c[-3] + ' ' + c[-2]
    row_example['precise_location'] = c
    row_example.drop(['area'], axis = 1, inplace = True)
    return row_example
row_example = pre_processing_precise_location(row_example)

In [45]:
row_example.head()

,price_per_m2,asking_price,price_evolution,typeProperty,typeOfForm,rooms,size,bisize,balkong,tomtarea,buildArea,cost,uteplats,floor,type,precise_location
4,NaN,2 395 000 kr,-95 000 kr (-4 %),Vinterbonat fritidshus,Äganderätt,2 rum,45 m²,7 m²,NaN,920 m²,1910,32 034 kr/år,NaN,NaN,bostadsratt,"Södersvik, Norrtälje kommun"


In [46]:
def pre_processing_distance_from_city_center(row_example):    
    row_example['precise_location'] 
    row_example['distance_from_city_center'] = ''
    i = 0
    #divide row_example dataframe in 25 different dataframes
    a = calculate_distance(row_example['precise_location'].iloc[i], 'Stockholm, Sweden')
    if(a != None):
        row_example['distance_from_city_center'].iloc[i] = a
    if(a == None):
        a = calculate_distance(row_example['precise_location'].iloc[i], 'Stockholm, Sweden')
        if(a != None):
            row_example['distance_from_city_center'].iloc[i] = a
        if(a == None):
            d = row_example['precise_location'].iloc[i]
            d = d.replace('/', ' ')
            a = calculate_distance(d, 'Stockholm, Sweden')
            if(a != None):
                row_example['distance_from_city_center'].iloc[i] = a
            if(a == None):
                d = d.split(' ')
                x = ''
                for c in range(len(d)):
                    if c == 0:
                        continue
                    x = x + d[c] + ' '
                a = calculate_distance(x, 'Stockholm, Sweden')
                if(a != None):
                    row_example['distance_from_city_center'].iloc[i] = a
                elif(a == None):
                    a = row_example['precise_location'].iloc[i]
                    a = a.split(' ')
                    a = a[-2:]
                    a = a[0] + ' ' + a[1]
                    distance = calculate_distance(a, 'Stockholm, Sweden')
                    print('iteration' + ' ' , i)
                    if(distance == None):
                        row_example['distance_from_city_center'].iloc[i] = 'not found'
                    else:
                        row_example['distance_from_city_center'].iloc[i] = distance
    return row_example
row_example = pre_processing_distance_from_city_center(row_example)

row_example.head()
                

,price_per_m2,asking_price,price_evolution,typeProperty,typeOfForm,rooms,size,bisize,balkong,tomtarea,buildArea,cost,uteplats,floor,type,precise_location,distance_from_city_center
4,NaN,2 395 000 kr,-95 000 kr (-4 %),Vinterbonat fritidshus,Äganderätt,2 rum,45 m²,7 m²,NaN,920 m²,1910,32 034 kr/år,NaN,NaN,bostadsratt,"Södersvik, Norrtälje kommun",67.264689


In [47]:
print(row_example)

  price_per_m2  asking_price    price_evolution            typeProperty  \
4          NaN  2 395 000 kr  -95 000 kr (-4 %)  Vinterbonat fritidshus   

   typeOfForm  rooms   size bisize balkong tomtarea buildArea          cost  \
4  Äganderätt  2 rum  45 m²   7 m²     NaN   920 m²      1910  32 034 kr/år   

  uteplats floor         type             precise_location  \
4      NaN   NaN  bostadsratt  Södersvik, Norrtälje kommun   

  distance_from_city_center  
4                 67.264689  


In [48]:
# change -2 600 000 kr to -2600000
from numpy import double

i = 0
def changePriceEvolution(x):
    x = x.split('(')
    x[1] = x[1].replace('\xa0','')
    x[1] = x[1].replace(')','')
    x[1] = x[1].replace('%','')
    x[0] = x[0].replace('\xa0','')
    x[0] = x[0].replace('kr','')
    x[1] = float(x[1])/100
    x[0] = float(x[0]) / 1000000
    return x

row_example['price_evolution_numbers(per_million)'] = ''
row_example['price_evolution_percentage(comma)'] = ''
row_example['price_evolution_numbers(per_million)'].iloc[i] = changePriceEvolution(row_example['price_evolution'].iloc[i])[0]
row_example['price_evolution_percentage(comma)'].iloc[i] = changePriceEvolution(row_example['price_evolution'].iloc[i])[1]
row_example.drop(['price_evolution'], axis = 1, inplace = True)

In [49]:
def pre_processing_asking_price(row_example):
    i = 0
    a = row_example['asking_price'].iloc[i]
    a = a.replace('\xa0','')
    a = a.replace('kr','')
    row_example['asking_price'].iloc[i] = float(a)
    return row_example

row_example = pre_processing_asking_price(row_example)
row_example['asking_price(per_million)'] = ''
row_example['asking_price(per_million)'] = row_example['asking_price'] / 1000000
row_example.drop(['asking_price'],axis=1,inplace=True)
row_example

,price_per_m2,typeProperty,typeOfForm,rooms,size,bisize,balkong,tomtarea,buildArea,cost,uteplats,floor,type,precise_location,distance_from_city_center,price_evolution_numbers(per_million),price_evolution_percentage(comma),asking_price(per_million)
4,NaN,Vinterbonat fritidshus,Äganderätt,2 rum,45 m²,7 m²,NaN,920 m²,1910,32 034 kr/år,NaN,NaN,bostadsratt,"Södersvik, Norrtälje kommun",67.264689,-0.095,-0.04,2.395


In [50]:
#read lookup table
lookup_table = pd.read_csv("./Data/lookup_table_typeProperty.csv")

def pre_processing_typeProperty(row_example):
    i = 0
    a = row_example['typeProperty'].iloc[i]
    row_example['typeProperty'].iloc[i] = lookup_table[lookup_table['typeProperty'] == a]['code'].iloc[0]
    return row_example

row_example = pre_processing_typeProperty(row_example)
row_example
lookup_table = pd.read_csv("./Data/lookup_table_typeOfForm.csv")
def pre_rpocessing_typeOfForm(row_example):
    i = 0
    a = row_example['typeOfForm'].iloc[i]
    row_example['typeOfForm'].iloc[i] = lookup_table[lookup_table['typeOfForm'] == a]['code'].iloc[0]
    return row_example
row_example = pre_rpocessing_typeOfForm(row_example)
lookup_table = pd.read_csv("./Data/lookup_table_type.csv")
def pre_processing_type(row_example):
    i = 0
    a = row_example['type'].iloc[i]
    row_example['type'].iloc[i] = lookup_table[lookup_table['type'] == a]['code'].iloc[0]
    return row_example
row_example = pre_processing_type(row_example)

In [51]:
row_example

,price_per_m2,typeProperty,typeOfForm,rooms,size,bisize,balkong,tomtarea,buildArea,cost,uteplats,floor,type,precise_location,distance_from_city_center,price_evolution_numbers(per_million),price_evolution_percentage(comma),asking_price(per_million)
4,NaN,9,5,2 rum,45 m²,7 m²,NaN,920 m²,1910,32 034 kr/år,NaN,NaN,1,"Södersvik, Norrtälje kommun",67.264689,-0.095,-0.04,2.395


In [52]:
def pre_processing_rooms(row_example):
    i = 0
    b = 0
    c = row_example['rooms'].iloc[i]
    # if c is an integer then do nothing
    if isinstance(c, int):
        b = 1
    else:
        c = c.replace(',','.')
        c = c.replace('rum','')
    row_example['rooms'].iloc[i] = float(c)
    return row_example
row_example = pre_processing_rooms(row_example)

In [53]:
row_example

,price_per_m2,typeProperty,typeOfForm,rooms,size,bisize,balkong,tomtarea,buildArea,cost,uteplats,floor,type,precise_location,distance_from_city_center,price_evolution_numbers(per_million),price_evolution_percentage(comma),asking_price(per_million)
4,NaN,9,5,2.0,45 m²,7 m²,NaN,920 m²,1910,32 034 kr/år,NaN,NaN,1,"Södersvik, Norrtälje kommun",67.264689,-0.095,-0.04,2.395


In [54]:

def pre_processing_size(row_example):
    a = row_example['size'].iloc[i]
    if isinstance(a, int):
        b = 0
    else:
        a = a.replace(',','.')
        a = a.replace('\xa0','')
        a = a.replace('m²','')
    row_example['size'].iloc[i] = float(a)
    return row_example
row_example = pre_processing_size(row_example)

In [55]:
row_example

,price_per_m2,typeProperty,typeOfForm,rooms,size,bisize,balkong,tomtarea,buildArea,cost,uteplats,floor,type,precise_location,distance_from_city_center,price_evolution_numbers(per_million),price_evolution_percentage(comma),asking_price(per_million)
4,NaN,9,5,2.0,45.0,7 m²,NaN,920 m²,1910,32 034 kr/år,NaN,NaN,1,"Södersvik, Norrtälje kommun",67.264689,-0.095,-0.04,2.395


In [56]:
def pre_processing_tomtarea(row_example):
    i = 0
    a = row_example['tomtarea'].iloc[i]
    print(a)
    if isinstance(a, float):
        b = 0
    elif isinstance(a, int):
        b = 0
    elif('ha' in a):
        a = a.replace(',','.')
        a = a.replace('\xa0','')
        a = a.replace('ha','')
        a = float(a) * 10000
    else:
        a = a.replace(',','.')
        a = a.replace('\xa0','')
        a = a.replace(' ','')
        a = a.replace('m²','')
        print(i)
    row_example['tomtarea'].iloc[i] = float(a)
    return row_example
row_example = pre_processing_tomtarea(row_example)

920 m²
0


In [191]:
row_example['size'] = row_example['size'] / 100
row_example['tomtarea'] = row_example['tomtarea'] / 10000

In [57]:
row_example

,price_per_m2,typeProperty,typeOfForm,rooms,size,bisize,balkong,tomtarea,buildArea,cost,uteplats,floor,type,precise_location,distance_from_city_center,price_evolution_numbers(per_million),price_evolution_percentage(comma),asking_price(per_million)
4,NaN,9,5,2.0,45.0,7 m²,NaN,920.0,1910,32 034 kr/år,NaN,NaN,1,"Södersvik, Norrtälje kommun",67.264689,-0.095,-0.04,2.395


In [58]:
def pre_processing_bisize(row_example):
    i = 0
    a = row_example['bisize'].iloc[i]
    if isinstance(a, float):
        b = 0
    elif isinstance(a, int):
        b = 0
    elif('ha' in a):
        a = a.replace(',','.')
        a = a.replace('\xa0','')
        a = a.replace('ha','')
        a = float(a) * 10000
    else:
        a = a.replace(',','.')
        a = a.replace('\xa0','')
        a = a.replace(' ','')
        a = a.replace('m²','')
    row_example['bisize'].iloc[i] = float(a)
    return row_example
row_example = pre_processing_bisize(row_example)

In [59]:
row_example['bisize'] = row_example['bisize'] / 100

In [60]:
row_example

,price_per_m2,typeProperty,typeOfForm,rooms,size,bisize,balkong,tomtarea,buildArea,cost,uteplats,floor,type,precise_location,distance_from_city_center,price_evolution_numbers(per_million),price_evolution_percentage(comma),asking_price(per_million)
4,NaN,9,5,2.0,45.0,0.07,NaN,920.0,1910,32 034 kr/år,NaN,NaN,1,"Södersvik, Norrtälje kommun",67.264689,-0.095,-0.04,2.395


In [61]:
def preprocessing_cost(row_example):
    a = row_example['cost'].iloc[i]
    if isinstance(a, float):
        b = 0
    elif isinstance(a, int):
        b = 0
    else:
        a = a.replace(',','.')
        a = a.replace('\xa0','')
        a = a.replace('kr/år','')
    row_example['cost'].iloc[i] = float(a)
    return row_example
row_example = preprocessing_cost(row_example)

In [62]:
row_example['cost_per_year(x1000)'] = ''
row_example['cost_per_year(x1000)'] = row_example['cost']

In [63]:
row_example['cost_per_year(x1000)'] = row_example['cost_per_year(x1000)'] / 1000

In [64]:
row_example.drop(['cost'],axis=1,inplace=True)

In [65]:
def pre_processing_buildarea(row_example):
    e = row_example['buildArea'].iloc[i]
    if isinstance(e, int):
        b = 0
    elif isinstance(e, float):
        b = 0
    for j in range(1900, 2024):
        intermediate = str(j)
        a = 0
        if intermediate in e:
            e = str(j)
            e = float(e)
            a = 1
            break
    if (a == 0):
        for k in range(0, 100):
            intermediate = str(k)
            if intermediate in e:
                if k < 50:
                    e = '19' + '0' + str(k)
                    e = float(e)
                    break
                else:
                    e = '19' + str(k)
                    e = float(e)
                    break
    if isinstance(e, str):
        e = 0
    row_example['buildArea'].iloc[i] = float(e)
    return row_example
row_example = pre_processing_buildarea(row_example)

In [66]:
row_example['buildArea'] = row_example['buildArea'] / 1000

In [67]:
row_example

,price_per_m2,typeProperty,typeOfForm,rooms,size,bisize,balkong,tomtarea,buildArea,uteplats,floor,type,precise_location,distance_from_city_center,price_evolution_numbers(per_million),price_evolution_percentage(comma),asking_price(per_million),cost_per_year(x1000)
4,NaN,9,5,2.0,45.0,0.07,NaN,920.0,1.91,NaN,NaN,1,"Södersvik, Norrtälje kommun",67.264689,-0.095,-0.04,2.395,32.034


In [68]:
row_example.rename(columns={'size': 'size(x100)'}, inplace=True)

In [69]:
row_example.rename(columns={'bisize': 'bisize(x100)'}, inplace=True)

In [70]:
row_example.rename(columns={'tomtarea': 'tomtarea(x10000)'}, inplace=True)

In [72]:
row_example.rename(columns={'year': 'year(x1000)'}, inplace=True)   

In [73]:
row_example

,price_per_m2,typeProperty,typeOfForm,rooms,size(x100),bisize(x100),balkong,tomtarea(x10000),buildArea,uteplats,floor,type,precise_location,distance_from_city_center,price_evolution_numbers(per_million),price_evolution_percentage(comma),asking_price(per_million),cost_per_year(x1000)
4,NaN,9,5,2.0,45.0,0.07,NaN,920.0,1.91,NaN,NaN,1,"Södersvik, Norrtälje kommun",67.264689,-0.095,-0.04,2.395,32.034


In [74]:
row_example.fillna(0, inplace=True)


In [75]:
i = 0
if((row_example['size(x100)']).iloc[i] != 0) or (row_example['bisize(x100)'].iloc[i] != 0):
    row_example['price_per_m2'].iloc[i] = row_example['asking_price(per_million)'].iloc[i]*1000000 / (row_example['size(x100)'].iloc[i]*100 + row_example['bisize(x100)'].iloc[i]*100)

In [76]:
row_example

,price_per_m2,typeProperty,typeOfForm,rooms,size(x100),bisize(x100),balkong,tomtarea(x10000),buildArea,uteplats,floor,type,precise_location,distance_from_city_center,price_evolution_numbers(per_million),price_evolution_percentage(comma),asking_price(per_million),cost_per_year(x1000)
4,531.395607,9,5,2.0,45.0,0.07,0,920.0,1.91,0,0,1,"Södersvik, Norrtälje kommun",67.264689,-0.095,-0.04,2.395,32.034


In [77]:
row_example['price_per_m2(x10000)'] = ''
row_example['price_per_m2(x10000)'] = row_example['price_per_m2'] / 10000

In [78]:
row_example.drop(['price_per_m2'],axis=1,inplace=True)

In [79]:
def pre_processing_uteplats(row_example):
    a = row_example['uteplats'].iloc[i]
    if (a != 0):
        a = 1
    row_example['uteplats'].iloc[i] = a
    return row_example
row_example = pre_processing_uteplats(row_example)


In [80]:
def pre_processing_balkong(row_example):
    a = row_example['balkong'].iloc[i]
    if (a != 'Nej'):
        a = 1
    else:
        a = 0
    row_example['balkong'].iloc[i] = a
    return row_example
row_example = pre_processing_balkong(row_example)

In [88]:
row_example

,typeProperty,typeOfForm,rooms,size(x100),bisize(x100),balkong,tomtarea(x10000),buildArea,uteplats,floor,type,distance_from_city_center,price_evolution_numbers(per_million),asking_price(per_million),cost_per_year(x1000),price_per_m2(x10000),elevator
4,9,5,2.0,45.0,0.07,1,920.0,1.91,0,0,1,67.264689,-0.095,2.395,32.034,0.05314,0


In [82]:
def pre_processing_elevator(row_example):
    row_example['elevator'] = ''
    i = 0
    for i in range(len(row_example['floor'])):
        a = row_example['floor'].iloc[i]
        if isinstance(a, int):
            continue
        elif isinstance(a, float):
            continue
        elif isinstance(a, np.int64):
            continue
        if(('hiss finns ej' in a) or ('hiss fins ej' in a)):
            a = 0
            row_example['elevator'].iloc[i] = a
            a = row_example['floor'].iloc[i].replace('hiss finns ej','')
            row_example['floor'].iloc[i] = a
        elif('hiss finns' in a) or ('hiss fins' in a):
            a = 1
            row_example['elevator'].iloc[i] = a
            a = row_example['floor'].iloc[i].replace('hiss finns','')
            row_example['floor'].iloc[i] = a
        else:
            a = 0
            row_example['elevator'].iloc[i] = a
    return row_example
row_example = pre_processing_elevator(row_example)


In [83]:
row_example

,typeProperty,typeOfForm,rooms,size(x100),bisize(x100),balkong,tomtarea(x10000),buildArea,uteplats,floor,type,precise_location,distance_from_city_center,price_evolution_numbers(per_million),price_evolution_percentage(comma),asking_price(per_million),cost_per_year(x1000),price_per_m2(x10000),elevator
4,9,5,2.0,45.0,0.07,1,920.0,1.91,0,0,1,"Södersvik, Norrtälje kommun",67.264689,-0.095,-0.04,2.395,32.034,0.05314,


In [84]:
i = 0
for i in range(len(row_example['floor'])):
    a = row_example['floor'].iloc[i]
    if isinstance(a, int):
        continue
    elif isinstance(a, float):
        continue
    elif isinstance(a, np.int64):
        continue
    c = a.split('av')
    c[0] = c[0].replace(',','')
    if(len(c) == 2):
        c[1] = c[1].replace(',','')
        a = float(c[0])
        b = float(c[1])
        a = a / b
        a = float(a)
    row_example['floor'].iloc[i] = a

In [85]:
#remove any commas or strings from the whole dataframe
row_example = row_example.replace(',','.', regex=True)
row_example = row_example.replace(' ','', regex=True)
row_example = row_example.replace('m²','', regex=True)
row_example = row_example.replace('kr','', regex=True)
row_example = row_example.replace('rum','', regex=True)
row_example = row_example.replace('ha','', regex=True)
row_example = row_example.replace('år','', regex=True)
row_example = row_example.replace('av','', regex=True)
row_example = row_example.replace('hiss finns ej','', regex=True)
row_example = row_example.replace('hiss finns','', regex=True)
row_example = row_example.replace('hiss fins ej','', regex=True)
row_example = row_example.replace('hiss fins','', regex=True)
row_example = row_example.replace('Nej','', regex=True)
row_example = row_example.replace('Ja','', regex=True)

#change all strings to zeros
row_example = row_example.replace('',0, regex=True)

row_example.drop(['price_evolution_percentage(comma)'],axis=1,inplace=True)
#remove everything that is string
row_example = row_example.apply(pd.to_numeric, errors='coerce')
#filling all the nan values with zeros
row_example = row_example.fillna(0)
row_example.drop(['precise_location'],axis=1,inplace=True)

In [86]:
#print number of columns
row_example.columns

Index(['typeProperty', 'typeOfForm', 'rooms', 'size(x100)', 'bisize(x100)',
       'balkong', 'tomtarea(x10000)', 'buildArea', 'uteplats', 'floor', 'type',
       'distance_from_city_center', 'price_evolution_numbers(per_million)',
       'asking_price(per_million)', 'cost_per_year(x1000)',
       'price_per_m2(x10000)', 'elevator'],
      dtype='object')

In [89]:
#save the row_example dataframe to a csv file  
row_example.to_csv('./Data//row_example.csv', index = False)

In [90]:
row_example

,typeProperty,typeOfForm,rooms,size(x100),bisize(x100),balkong,tomtarea(x10000),buildArea,uteplats,floor,type,distance_from_city_center,price_evolution_numbers(per_million),asking_price(per_million),cost_per_year(x1000),price_per_m2(x10000),elevator
4,9,5,2.0,45.0,0.07,1,920.0,1.91,0,0,1,67.264689,-0.095,2.395,32.034,0.05314,0
